In [1]:
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation

from explaneat.core.neuralneat import NeuralNeat as NeuralNeat

import random

import torch
import torch.nn as nn
import torch.optim as optim

import neat

from explaneat.core.utility import MethodTimer


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter

from explaneat.core.neuralneat import NeuralNeat as nneat

import torch.optim as optim

from torch.functional import F








In [2]:
RANDOM_SEED      = 42
torch.manual_seed(RANDOM_SEED)


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [3]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [4]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    with MethodTimer("Backprop everything"):
        p = BackpropPopulation(config, 
                                xs, 
                                ys, 
                                criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

config = base_config
saveLocation = './'
maxNGenerations = 2
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = 5)

g = p.best_genome


The function - Backprop everything - has just started at 1635034191.724975
The function - Backprop everything - took 0.003513813018798828 seconds to complete
The function - generationStart - has just started at 1635034191.728925

 ****** Running generation 0 ****** 

The function - generationStart - took 6.29425048828125e-05 seconds to complete
The function - pre_backprop - has just started at 1635034191.729216
The function - pre_backprop - took 0.00010895729064941406 seconds to complete
The function - backprop - has just started at 1635034191.729345
about to start backprop with 5 epochs
PRE
DefaultConnectionGene(key=(-1, 0), weight=-1.657550337258934, enabled=True)
DefaultConnectionGene(key=(-2, 0), weight=0.8390453351650665, enabled=True)
DefaultConnectionGene(key=(-1, 0), weight=-1.6737605175320156, enabled=True)
DefaultConnectionGene(key=(-2, 0), weight=0.8552706145672907, enabled=True)
PRE
DefaultConnectionGene(key=(-1, 0), weight=-2.5433298777819338, enabled=True)
DefaultConnecti

In [5]:
print(g)

Key: 140
Fitness: 1.3882861137390137
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.0996119245961289, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=0.026545819578642375, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.04349980025769255, enabled=True)


In [6]:
net = nneat(g, p.config, criterion=nn.BCEWithLogitsLoss())


In [7]:
net.genome.connections
# Error is in inputs not being updated with extra info from skip layers

{(-1, 0): <neat.genes.DefaultConnectionGene at 0x7fbb380c5550>,
 (-2, 0): <neat.genes.DefaultConnectionGene at 0x7fbb380c55b0>}

In [8]:
net.forward(torch.tensor( xor_inputs[:10], dtype=torch.float64) ) [:10]

tensor([[0.4784],
        [0.4771],
        [0.4653],
        [0.4793],
        [0.4642],
        [0.4725],
        [0.4716],
        [0.4748],
        [0.4695],
        [0.4795]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [9]:
net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )[:20]

tensor([[0.4784],
        [0.4771],
        [0.4653],
        [0.4793],
        [0.4642],
        [0.4725],
        [0.4716],
        [0.4748],
        [0.4695],
        [0.4795],
        [0.4729],
        [0.4819],
        [0.4740],
        [0.4695],
        [0.4843],
        [0.4842],
        [0.4741],
        [0.4809],
        [0.4669],
        [0.4850]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [10]:
# net.optimiser.zero_grad()

In [11]:
# net.optimise(xor_inputs, xor_outputs)

In [12]:
optimiser = optim.Adadelta(net.parameters())

In [13]:
net.weights

{1: Parameter containing:
 tensor([[-0.0435],
         [ 0.0265]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [14]:
optimizer = optimiser
losses = []
for i in range(10000):
    preds = net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )
    loss = F.mse_loss(preds, torch.tensor( xor_outputs, dtype=torch.float64)).sqrt()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss)  
losses[-10:]


[tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>),
 tensor(0.4996, dtype=torch.float64, grad_fn=<SqrtBackward0>)]

In [15]:
preds[:10]

tensor([[0.4731],
        [0.4822],
        [0.4902],
        [0.4888],
        [0.4984],
        [0.4744],
        [0.4587],
        [0.4828],
        [0.4655],
        [0.4901]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [16]:
net.weights

{1: Parameter containing:
 tensor([[-0.0728],
         [-0.0884]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [17]:
net.layers

{1: {'nodes': {0: {'depth': 1,
    'output_ids': [],
    'input_ids': [-1, -2],
    'output_layers': [],
    'needs_skip': False,
    'id': 0,
    'input_layers': [0, 0],
    'skip_layer_input': False,
    'layer_index': 0}},
  'is_output_layer': True,
  'is_input_layer': False,
  'layer_type': 'OUTPUT',
  'input_layers': [0],
  'input_shape': 2,
  'weights_shape': (2, 1),
  'out_weights': [],
  'bias': [-0.0996119245961289],
  'in_weights': [[0, 0]],
  'input_weights': array([[-0.0434998 ],
         [ 0.02654582]]),
  'input_map': {(-1, 0): (0, 0), (-2, 0): (1, 0)}},
 0: {'nodes': {-1: {'depth': 0,
    'output_ids': [0],
    'input_ids': [],
    'output_layers': [1],
    'needs_skip': False,
    'id': -1,
    'input_layers': [],
    'skip_layer_input': False,
    'layer_index': 0},
   -2: {'depth': 0,
    'output_ids': [0],
    'input_ids': [],
    'output_layers': [1],
    'needs_skip': False,
    'id': -2,
    'input_layers': [],
    'skip_layer_input': False,
    'layer_index': 1}}

In [18]:
net.node_tracker[0]['depth']

1

In [19]:


reached_nodes = []

node_stack = []

# Instantiate stack with depth==0 nodes
for node_id, node in net.node_tracker.items():
    if node_id < 0:
        node_stack.append(node_id)

while len(node_stack) > 0:
    reached_nodes.append(node_stack[0])
    for node_id in net.node_tracker[node_stack[0]]['output_ids']:
        node_stack.append(node_id)
    del(node_stack[0])

if True:
    print("the reached nodes are")
    print(reached_nodes)


for node_id in net.node_tracker:
    if not node_id in reached_nodes:
        print(reached_nodes)
        print(net.node_tracker)
        print("I can't reach this node {}".format(node_id))

the reached nodes are
[-1, -2, 0, 0]


In [20]:
print(net.genome)

Key: 140
Fitness: 1.3882861137390137
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.0996119245961289, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=0.026545819578642375, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.04349980025769255, enabled=True)
